In [3]:
from dc_pue.core.simulation_funs_DC import PUE_WUE_WE_Chiller_Colo

In [ ]:
# Step 2: specify user inputs for the PUE& WUE calulator: 
# you can refer to Table B.1 in our RCR paper, or comments in "simulation_funs_DC.py"
# you can also specify your customerized inputs
x = [ 
    9,                      # T_oa: Ambient temperature (deg C)
    10,                     # RH_oa: Ambient relative humidity (%)   
    101325,                 # P_oa: Ambient pressure (Pa)
    8.94746094e-01,         # UPS_e: UPS efficiency relative to IT power (relative): P_loss = (Power_IT/UPS_e - Power_IT)
    2.84082031e-02,         # PD_lr: distribution loss relative to IT power (relative): P_loss = (Power_IT/(1-PD_lr) - Power_IT)   
    4.72167969e-02,         # L_percentage = lighting power relative to IT power (relative): P_loss = (Power_IT*L_percentage)
    9.51835937e-01,         # SHR: sensible heat ratio (relative): Q_s/(Q_s+Q_l)
    9.49707031e+00,         # delta_T_air: temperature difference supply and return CRAH (C)
    6.50488281e+02,         # Fan_Pressure_CRAC: fan pressure CRAH - cooling tower (Pa)
    6.13867188e-01,         # Fan_e_CRAC: fan efficiency - cooling tower (relative)
    6.41894531e+06,         # Pump_Pressure_HD: pump pressure - humidification (Pa) 
    6.50976563e-01,         # Pump_e_HD: pump efficiency - humidification (relative)
    6.90283203e-01,         # HTE: heat exchanger effectiveness (relative)
    6.91894531e+00,         # delta_T_water; difference in facility water temperature (C) = T_return-T_supply 
    3.35986328e+00,         # AT_CT: approach temperature cooling tower (C)
    2.10712891e+00,         # AT_HE: approach temperature economizer heat exchanger
    1.56284191e+05,         # Pump_Pressure_WE: pump pressure waterside economizer (Pa)
    6.68945312e-01,         # Pump_e_WE: pump efficiency economizer (relative)  
    1.71884552e+05,         # Pump_Pressure_CW: pump pressure chiller pump (Pa) 
    7.45117188e-01,         # Pump_e_CW: Pump efficiency chiller pump (relative)
    3.08984375e-01,         # Chiller_load: Chiller partial load factor (relative)
    5.09960938e+00,         # delta_T_CT: temperature difference cooling tower (C): T_return-T_supply 
    2.46235102e+05,         # Pump_Pressure_CT: pump pressure cooling tower
    7.08007812e-01,         # Pump_e_CT: pump efficiency cooling tower
    2.93588867e-03,         # Windage_p: windage loss of water relative to cooling tower (relative)
    1.11650391e+01,         # CC: cooling tower cycles of concentration
    2.72070313e-01,         # LGRatio: liquid-gas ratio cooling tower
    3.95898438e+02,         # Fan_Pressure_CT: fan pressure cooling tower (Pa)
    6.11914063e-01,         # Fan_e_CT: fan pressure efficiency (relative)
    2.87041016e+01,         # T_up: Supply air dry bulb high setpoint (C)
    1.54423828e+01,         # T_lw: Supply air dry bulb low setpoint (C)
    1.67460938e+01,         # dp_up: Supply air dew point high setpoint (C)
    -1.11123047e+01,        # dp_lw: Supply air dew point low setpoint (C)
    7.18554688e+01,         # RH_up: Supply air relative high humidity setpoint (relative)
    2.60351562e+01,         # RH_lw: Supply air relative low humidity setpoint (relative)
    -9.17968750e-02         # pcop: COP relative error to regressed value
]

In [ ]:
def parse_model_input(data_dict):
    """
    Parse a dictionary of inputs and return a list of values
    ordered according to the expected input structure.
    """

    ordered_keys = [
        "T_oa", "RH_oa", "P_oa", "UPS_e", "PD_lr", "L_percentage",
        "SHR", "delta_T_air", "Fan_Pressure_CRAC", "Fan_e_CRAC",
        "Pump_Pressure_HD", "Pump_e_HD", "HTE", "delta_T_water",
        "AT_CT", "AT_HE", "Pump_Pressure_WE", "Pump_e_WE",
        "Pump_Pressure_CW", "Pump_e_CW", "Chiller_load",
        "delta_T_CT", "Pump_Pressure_CT", "Pump_e_CT", "Windage_p",
        "CC", "LGRatio", "Fan_Pressure_CT", "Fan_e_CT",
        "T_up", "T_lw", "dp_up", "dp_lw", "RH_up", "RH_lw", "pcop"
    ]

    # Optional: Check for missing keys
    missing_keys = [key for key in ordered_keys if key not in data_dict]
    if missing_keys:
        raise ValueError(f"Missing keys in input dictionary: {missing_keys}")

    return [data_dict[key] for key in ordered_keys]

In [ ]:
input_metadata = {
    "T_oa": {
        "description": "Ambient temperature",
        "units": "C"
    },
    "RH_oa": {
        "description": "Ambient relative humidity",
        "units": "%"
    },
    "P_oa": {
        "description": "Ambient pressure",
        "units": "Pa"
    },
    "UPS_e": {
        "description": "UPS efficiency relative to IT power",
        "units": "relative"
    },
    "PD_lr": {
        "description": "Distribution loss relative to IT power",
        "units": "relative"
    },
    "L_percentage": {
        "description": "Lighting power relative to IT power",
        "units": "relative"
    },
    "SHR": {
        "description": "Sensible heat ratio (Q_s / (Q_s + Q_l))",
        "units": "relative"
    },
    "delta_T_air": {
        "description": "Temperature difference between CRAH supply and return",
        "units": "C"
    },
    "Fan_Pressure_CRAC": {
        "description": "CRAH fan pressure",
        "units": "Pa"
    },
    "Fan_e_CRAC": {
        "description": "CRAH fan efficiency",
        "units": "relative"
    },
    "Pump_Pressure_HD": {
        "description": "Pump pressure for humidification",
        "units": "Pa"
    },
    "Pump_e_HD": {
        "description": "Pump efficiency for humidification",
        "units": "relative"
    },
    "HTE": {
        "description": "Heat exchanger effectiveness",
        "units": "relative"
    },
    "delta_T_water": {
        "description": "Facility water temperature difference (return - supply)",
        "units": "C"
    },
    "AT_CT": {
        "description": "Cooling tower approach temperature",
        "units": "C"
    },
    "AT_HE": {
        "description": "Economizer heat exchanger approach temperature",
        "units": "C"
    },
    "Pump_Pressure_WE": {
        "description": "Pump pressure waterside economizer",
        "units": "Pa"
    },
    "Pump_e_WE": {
        "description": "Pump efficiency waterside economizer",
        "units": "relative"
    },
    "Pump_Pressure_CW": {
        "description": "Pump pressure chiller pump",
        "units": "Pa"
    },
    "Pump_e_CW": {
        "description": "Pump efficiency chiller pump",
        "units": "relative"
    },
    "Chiller_load": {
        "description": "Chiller partial load factor",
        "units": "relative"
    },
    "delta_T_CT": {
        "description": "Cooling tower temperature difference (return - supply)",
        "units": "C"
    },
    "Pump_Pressure_CT": {
        "description": "Pump pressure cooling tower",
        "units": "Pa"
    },
    "Pump_e_CT": {
        "description": "Pump efficiency cooling tower",
        "units": "relative"
    },
    "Windage_p": {
        "description": "Windage water loss relative to cooling tower",
        "units": "relative"
    },
    "CC": {
        "description": "Cooling tower cycles of concentration",
        "units": "unitless"
    },
    "LGRatio": {
        "description": "Liquid-gas ratio in cooling tower",
        "units": "relative"
    },
    "Fan_Pressure_CT": {
        "description": "Cooling tower fan pressure",
        "units": "Pa"
    },
    "Fan_e_CT": {
        "description": "Cooling tower fan efficiency",
        "units": "relative"
    },
    "T_up": {
        "description": "Supply air dry bulb high setpoint",
        "units": "C"
    },
    "T_lw": {
        "description": "Supply air dry bulb low setpoint",
        "units": "C"
    },
    "dp_up": {
        "description": "Supply air dew point high setpoint",
        "units": "C"
    },
    "dp_lw": {
        "description": "Supply air dew point low setpoint",
        "units": "C"
    },
    "RH_up": {
        "description": "Supply air high relative humidity setpoint",
        "units": "relative"
    },
    "RH_lw": {
        "description": "Supply air low relative humidity setpoint",
        "units": "relative"
    },
    "pcop": {
        "description": "COP relative error to regressed value",
        "units": "relative"
    }
}


In [5]:
# Step 3: call the function to get PUE and WUE values

print(PUE_WUE_WE_Chiller_Colo(x))

(np.float64(1.3393524025151962), np.float64(2.417390377483526))
